<a href="https://colab.research.google.com/github/stavco9/nlp-final-project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/stavco9/nlp-final-project.git

Cloning into 'nlp-final-project'...
remote: Enumerating objects: 262, done.
remote: Counting objects: 100% (262/262), done.
remote: Compressing objects: 100% (170/170), done.
remote: Total 262 (delta 125), reused 178 (delta 67), pack-reused 0
Receiving objects: 100% (262/262), 9.10 MiB | 6.70 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [2]:
%cd /content/nlp-final-project/

/content/nlp-final-project


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using 

In [4]:
import os
import json
import pandas as pd
import numpy as np
import requests

In [5]:
# Mount Google Drive
from google.colab import drive, files
drive.mount('/content/drive')
!ln -s "/content/drive/My Drive/IDC_MSc/Year1/NLP/Final-Project/data" ./data

Mounted at /content/drive


In [ ]:
# Please set a valid Google API
# Please download the 'data' folder from https://drive.google.com/drive/folders/1D-OHX0LUNA5Y-0h8UXwqa4H_yETZN_t_ to your project root directory

GOOGLE_API_KEY = '**********************'

In [ ]:
##
## Dataset preperation part
##

In [ ]:
from modules.dataset.transcript_dataset import TranscriptDataset

dataset_builder = TranscriptDataset(api_key=GOOGLE_API_KEY)

# dataset_folder => The folder to load / save the datasets file from. If not exists it will create it automatically
# videos_count => Total number of videos to include in the dataset. If there are already saved videos in the "data" folder,
# it will fill the remaining videos count (For example if the parameter: videos_count is 4000 and there are already 1500 videos
# in the folder, it will fetch 2500 videos). Default is 100
# country_code => 2 digits ISO Code of the country you want to fetch videos from (If not set, the default is United States 'us')
# Relevant codes for countries with a majority of English based videos => 'ca' Canada, 'uk' United Kingdom, 'au' Australia, 'in' India
#  'nz' New Zealand, 'za' South Africa
# language => 2 digits ISO Code of the video's language you want to use (If not set, the default is English 'en')
dataset_builder.build_dataset(dataset_folder='data/dataset', videos_count=5000, country_code='uk')

# dataset_folder => The folder to load / save the datasets file from. If not exists it will create it automatically
# comments_count => The amount of comments to fetch for each video. Default is 100
dataset_builder.build_comments(dataset_folder='data', comments_count=200)

There are currnet 5000 videos in 10 files
There are already at least 5000. No need to do anything


In [ ]:
##
## Sentiment analysis part
##

In [ ]:
from modules.sentiment_analysis.comments_sentiment_analysis import multi_sentiment_score
from modules.sentiment_analysis.video_info import Video_info, datumToObj, save_to_json, save_to_pickle
from modules.sentiment_analysis.GPT_comment_controversy import init_openai, get_disagreement_rating
from modules.sentiment_analysis.scoring_video import adjust_disagreement_rating

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
# put all the json into pickle
datasets = [f'data/dataset/dataset_{i}.json' for i in range(10)]
to_print = True #input("Do you want to show prints? (y/n): ") != 'n'
basic_inf_vids = datumToObj(datasets, 'data/sentiment/videos_sample.pkl', to_print)
number_of_vids = len(basic_inf_vids)

save_to_json(basic_inf_vids, 'data/sentiment/vids_no_sentiment.json')
sentiment_model = multi_sentiment_score
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
count = 0
for vid in basic_inf_vids:
    print_text = "\r"
    if to_print:
        count += 1
        print_text += f"Processing video {count}/{number_of_vids}"
    vid.add_comments_scores(sentiment_model, model_name)
    if to_print:
        print_text += f" - Disagreement rating: {vid.comments_controversy_classical}"
    if to_print:
        print(print_text)

save_to_json(basic_inf_vids, 'data/sentiment/vids_classical_sentiment.json')
basic_inf_vids = adjust_disagreement_rating(basic_inf_vids)
save_to_json(basic_inf_vids, 'data/sentiment/videos_classical_sentiment_adjusted.json')
'''

#load vids from videos_classical_sentiment_adjusted.json and adjust basic_inf_vids with its values
with open('data/data/videos_classical_sentiment_adjusted.json') as json_file:
    json_obj = json.load(json_file)
for i, video_obj in enumerate(json_obj):
    basic_inf_vids[i].comments_controversy_classical = float(video_obj['comments_controversy_classical'])
'''


to_continue = True #input("Do you want to continue with GPT? (y/n): ")
if not to_continue:
  print("Done")
else:
  count = 0
  client = init_openai()
  sentiment_model=get_disagreement_rating
  model_name="GPT"
  for vid in basic_inf_vids:
      print_text = "\r"
      if to_print:
          count += 1
          print_text += f"Processing video {count}/{number_of_vids}"
      vid.add_comments_scores(sentiment_model, model_name, client)
      if to_print:
          print_text += f" - Disagreement rating: {vid.comments_controversy_GPT}"
      if to_print:
          print(print_text)
  save_to_json(basic_inf_vids, 'data/data/vids_GPT_sentiment.json')

In [ ]:
##
## Training part
##

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import random
import numpy as np
import json
import os
from modules.azsclm.azsclm import AZSC_LanguageModel
from modules.azsclm.board import Board
from transformers import AutoTokenizer

In [7]:
class Config_Model:
    def __init__(self, num_epochs, train_data, valid_data, test_data, model, optimizer, threshold_percentage, criterion, device):
        self.num_epochs = num_epochs
        self.train_data = train_data
        self.valid_data = valid_data
        self.test_data = test_data
        self.device = device
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.threshold_percentage = threshold_percentage


class Model:
    def __init__(self, config, data_name = 'GPT'):
        self.num_epochs = config.num_epochs
        self.train_data = config.train_data
        self.valid_data = config.valid_data
        self.test_data = config.test_data
        self.pred_score = []
        self.true_score = []
        self.device = config.device
        self.model = config.model
        self.optimizer = config.optimizer
        self.criterion = config.criterion
        self.data_name = 'comments_controversy_' + data_name
        self.max_available = 1
        self.min_available = -1
        self.board = Board()

    def calculate_accuracy(self, min_item, max_item, pred, true):
        pred = max(pred, self.min_available)
        pred = min(pred, self.max_available)
        true = max(true, self.min_available)
        true = min(true, self.max_available)

        acc = 1 - (abs(pred - true) / abs(max_item - min_item))
        return float(acc)

    def train(self):
        size = len(self.train_data.dataset)
        num_train_batches = len(self.train_data)
        num_valid_batches = len(self.valid_data)
        self.model.train()
        for epoch in range(self.num_epochs):
            print(f"Epoch {epoch+1}\n-------------------------------")
            epoch_loss = 0

            for batch_idx, (src, trg) in enumerate(self.train_data):
                src, trg = src.to(self.device), trg.to(self.device)
                output = self.model(src)

                self.optimizer.zero_grad()
                loss = self.criterion(output, trg)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.5)
                train_loss = loss.item()
                self.optimizer.step()
                epoch_loss += train_loss

            self.board.info_handler(loss=train_loss, batch=batch_idx, size=size, epoch_loss=epoch_loss, name='(training) ' + self.data_name)

            print("Evaluating with validation set")
            self.model.eval()
            running_valid_loss = 0
            total_acc = 0

            list_targets = [item[1].to('cuda') for item in self.valid_data]
            list_targets = torch.stack(list_targets, dim=1).to('cuda')
            max_item = min(torch.max(list_targets).item(), self.max_available)
            min_item = max(torch.min(list_targets).item(), self.min_available)

            # Disable gradient computation and reduce memory consumption.
            with torch.no_grad():
                for batch_idx, (src, trg) in enumerate(self.valid_data):
                    src, trg = src.to(self.device), trg.to(self.device)
                    output = self.model(src)
                    loss = self.criterion(output, trg)
                    valid_loss = loss.item()
                    running_valid_loss += valid_loss
                    total_acc += self.calculate_accuracy(max_item, min_item, output, trg)
                    avg_acc = total_acc / (batch_idx + 1)
                    print(f"Accuracy: {avg_acc}")

            avg_train_loss = epoch_loss / num_train_batches
            avg_valid_loss = running_valid_loss / num_valid_batches
            print(f"Average train loss: {avg_train_loss}, avarage valid loss: {avg_valid_loss}")
            avg_acc = total_acc / num_valid_batches
            print(f"Average accuracy rate: {avg_acc}")

        batch = next(iter(self.train_data))
        src = batch[0].to(self.device)
        self.board.add_graph(self.model, src)

    def test(self):
        size = len(self.test_data.dataset)
        num_batches = len(self.test_data)
        self.model.eval()
        epoch_loss = 0.0
        sections_losses = {}
        total_acc = 0

        list_targets = [item[1].to('cuda') for item in self.test_data]
        list_targets = torch.stack(list_targets, dim=1).to('cuda')
        max_item = min(torch.max(list_targets).item(), self.max_available)
        min_item = max(torch.min(list_targets).item(), self.min_available)
        with torch.no_grad():
            for batch_idx, (src, trg) in enumerate(self.test_data):
                src, trg = src.to(self.device), trg.to(self.device)
                pred = self.model(src)
                pred = pred.to(self.device)
                loss = self.criterion(pred, trg)
                #for section_name, section_loss in loss.last_losses.items():
                #    if section_name not in sections_losses:
                #        sections_losses[section_name] = torch.Tensor([0]).to(self.device)
                #    sections_losses[section_name] += section_loss
                #for section_name in sections_losses.keys():
                #    sections_losses[section_name] /= size
                epoch_loss = self.board.info_handler(loss=loss.item(), batch=batch_idx, size=size, epoch_loss=epoch_loss, name = '(test) ' + self.data_name)
                avg_loss = epoch_loss / num_batches
                print(f"Average loss: {avg_loss}")
                total_acc += self.calculate_accuracy(max_item, min_item, pred, trg)
                avg_acc = total_acc / (batch_idx + 1)
                print(f"Accuracy: {avg_acc}")
                self.pred_score.extend(pred.cpu().numpy())
                self.true_score.extend(trg.cpu().numpy())

        avg_loss = epoch_loss / num_batches
        print(f"Average loss rate: {avg_acc}")

        avg_acc = total_acc / num_batches
        print(f"Average accuracy rate: {avg_acc}")

    def calculate_metrics(self):
      # Convert lists to tensors for calculation
      true_score_tensor = torch.tensor(self.true_score)
      pred_score_tensor = torch.tensor(self.pred_score)

      # Calculating precision, recall, and F1 score using PyTorch
      TP = ((pred_score_tensor >= 0) & (true_score_tensor >= 0)).sum().item()
      FP = ((pred_score_tensor >= 0) & (true_score_tensor < 0)).sum().item()
      FN = ((pred_score_tensor < 0) & (true_score_tensor >= 0)).sum().item()

      precision = TP / (TP + FP) if TP + FP > 0 else 0
      recall = TP / (TP + FN) if TP + FN > 0 else 0
      f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

      print(f'Precision: {precision}')
      print(f'Recall: {recall}')
      print(f'F1 Score: {f1}')

      return (precision, recall, f1)



In [8]:
def custom_collate_fn(batch):
    src, tgt = zip(*batch)
    src = torch.stack(src).long()  # Ensure src is Long
    src = src.view(src.size(0), -1)
    tgt = torch.stack(tgt).float()  # Ensure tgt is Float
    tgt = tgt.view(tgt.size(0), -1)
    return src, tgt

class CustomDataset(Dataset):
    def __init__(self, texts, scores):
        self.texts = texts
        self.scores = scores

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        src = self.texts[idx]
        trg = self.scores[idx]
        return src, trg

def homogonize_token_length(text, tokenizer, text_length, device="cuda"):
    tokenized_text = tokenizer(text, return_tensors="pt", truncation=True, padding=True)['input_ids'].long().to(device)
    if tokenized_text.shape[1] < text_length:
        tokenized_text = torch.cat((tokenized_text, torch.zeros(tokenized_text.shape[0], text_length - tokenized_text.shape[1]).to(device)), dim=1)
    else:
        tokenized_text = tokenized_text[:, :text_length]
    return tokenized_text

class Datasets:
    def __init__(self, json_file, tokenizer, text_length, ratios=[0.8, 0.1, 0.1], batch_size=1, device="cuda"):
        self.json_file = json_file
        assert sum(ratios) == 1
        self.ratios = ratios
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.text_length = text_length
        self.device = device
        self.load_data()

    def load_data(self):
        with open(self.json_file, 'r') as f:
            data = json.load(f)

        random.shuffle(data)
        tokens = [homogonize_token_length(d[0], self.tokenizer, self.text_length, self.device) for d in data]
        GPT_scores = [torch.tensor(d[1], dtype=torch.float32).to(self.device) for d in data]
        BERT_scores = [torch.tensor(d[2], dtype=torch.float32).to(self.device) for d in data]

        print(BERT_scores)

        train_size = int(self.ratios[0] * len(data))
        val_size = int(self.ratios[1] * len(data))
        #test_size = len(data) - train_size - val_size

        train_text = tokens[:train_size]
        val_text = tokens[train_size:train_size + val_size]
        test_text = tokens[train_size + val_size:]

        train_scores1 = GPT_scores[:train_size]
        val_scores1 = GPT_scores[train_size:train_size + val_size]
        test_scores1 = GPT_scores[train_size + val_size:]

        train_scores2 = BERT_scores[:train_size]
        val_scores2 = BERT_scores[train_size:train_size + val_size]
        test_scores2 = BERT_scores[train_size + val_size:]

        train_dataset1 = CustomDataset(train_text, train_scores1)
        val_dataset1 = CustomDataset(val_text, val_scores1)
        test_dataset1 = CustomDataset(test_text, test_scores1)

        train_dataset2 = CustomDataset(train_text, train_scores2)
        val_dataset2 = CustomDataset(val_text, val_scores2)
        test_dataset2 = CustomDataset(test_text, test_scores2)

        # Create data loaders
        self.train_loader1 = DataLoader(train_dataset1, batch_size=self.batch_size, shuffle=True, collate_fn=custom_collate_fn)
        self.val_loader1 = DataLoader(val_dataset1, batch_size=self.batch_size, shuffle=False, collate_fn=custom_collate_fn)
        self.test_loader1 = DataLoader(test_dataset1, batch_size=self.batch_size, shuffle=False, collate_fn=custom_collate_fn)

        self.train_loader2 = DataLoader(train_dataset2, batch_size=self.batch_size, shuffle=True, collate_fn=custom_collate_fn)
        self.val_loader2 = DataLoader(val_dataset2, batch_size=self.batch_size, shuffle=False, collate_fn=custom_collate_fn)
        self.test_loader2 = DataLoader(test_dataset2, batch_size=self.batch_size, shuffle=False, collate_fn=custom_collate_fn)

In [9]:
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
    torch.cuda.empty_cache()

    print(torch.cuda.memory_summary(device="cuda", abbreviated=True))

    text_length = 4096
    tokenizer_name = 'distilbert-base-uncased-finetuned-sst-2-english'
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    datasets = Datasets("data/sentiment/vids_sentiment_fixed.json", tokenizer, text_length, ratios=[0.8, 0.1, 0.1], batch_size=1, device="cuda")

    # Instantiate the model
    model = AZSC_LanguageModel(tokenizer, text_length).to("cuda")

    # Instantiate the optimizer
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.1)

    # Instantiate the loss function
    criterion = nn.MSELoss()

    # the gpt model
    traind_data_gpt = datasets.train_loader1
    valid_data_gpt = datasets.val_loader1
    test_data_gpt = datasets.test_loader1

    config_gpt = Config_Model(num_epochs=2, train_data=traind_data_gpt, valid_data=valid_data_gpt, test_data=test_data_gpt, model=model, optimizer=optimizer, threshold_percentage = 5, criterion=criterion, device="cuda")
    # Instantiate the Train object
    model_gpt = Model(config_gpt, data_name='GPT')

    # the bert model
    trained_data_classical = datasets.train_loader2
    valid_data_classical = datasets.val_loader2
    test_data_classical = datasets.test_loader2
    config_classical = Config_Model(num_epochs=2, train_data=trained_data_classical, valid_data=valid_data_classical, test_data=test_data_classical, model=model, optimizer=optimizer, threshold_percentage = 5, criterion=criterion, device="cuda")
    # Instantiate the Train object
    model_classical = Model(config_classical, data_name='classical')


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Requested memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

[tensor(0.1883, device='cuda:0'), tensor(0.0771, device='cuda:0'), tensor(0.6210, device='cuda:0'), tensor(0.4659, device='cuda:0'), tensor(0.0152, device='cuda:0'), tensor(0.0119, device='cuda:0'), tensor(0.0300, device='cuda:0'), tensor(0.3040, device='cuda:0'), tensor(0.3163, device='cuda:0'), tensor(0.0025, device='cuda:0'), tensor(0.0158, device='cuda:0'), tensor(0.0259, device='cuda:0'), tensor(0.0102, device='cuda:0'), tensor(0.0014, device='cuda:0'), tensor(0.0100, device='cuda:0'), tensor(0.0062, device='cuda:0'), tensor(0.2027, device='cuda:0'), tensor(0.0002, device='cuda:0'), tensor(0.0093, device='cuda:0'), tensor(0.0043, device='cuda:0'), tensor(0.0004, device='cuda:0'), tensor(0.0034, device='cuda:0'), tensor(0.0381, device='cuda:0'), tensor(0.0009, device='cuda:0'), tensor(0.0115, device='cuda:0'), tensor(0.2968, device='cuda:0'), tensor(0.0021, device='cuda:0'), tensor(0.0375, device='cuda:0'), tensor(0.0015, device='cuda:0'), tensor(0.1129, device='cuda:0'), tensor(0.

In [12]:
#for batch_idx, (src, trg) in enumerate(test_data_classical):
#    src, trg = src.to('cuda'), trg.to('cuda')

src, trg = next(iter(test_data_gpt))
print(trg)
#targets = [item[1].to('cuda') for item in test_data_classical]
#targets[5]
#targets = torch.stack(targets, dim=1).to('cuda')
#max_item = min(torch.max(targets).item(), 1)
#print(type(max_item))
#min_item = max(torch.min(targets).item(), -1)
#print(type(min_item))
#print(abs(max_item - min_item) == 2)

tensor([[-0.6000]], device='cuda:0')


In [13]:
    # Start training
    model_classical.train()

Epoch 1
-------------------------------
Evaluating with validation set
Accuracy: 0.750238835811615
Accuracy: 0.7707732617855072
Accuracy: 0.7771401206652323
Accuracy: 0.780619278550148
Accuracy: 0.7830808520317077
Accuracy: 0.8098953862984976
Accuracy: 0.8074190786906651
Accuracy: 0.8185110092163086
Accuracy: 0.8187246653768752
Accuracy: 0.8158870816230774
Accuracy: 0.8293483744968068
Accuracy: 0.8261320342620214
Accuracy: 0.8126071370564975
Accuracy: 0.8231022017342704
Accuracy: 0.8228257417678833
Accuracy: 0.821480643004179
Accuracy: 0.8198023578699898
Accuracy: 0.8217804133892059
Accuracy: 0.8242052385681554
Accuracy: 0.8225781440734863
Accuracy: 0.821604345526014
Accuracy: 0.8204217959534038
Accuracy: 0.819088762221129
Accuracy: 0.8203447461128235
Accuracy: 0.8212646150588989
Accuracy: 0.82015460729599
Accuracy: 0.8198553632806849
Accuracy: 0.8026412810598101
Accuracy: 0.803013840626026
Accuracy: 0.8069941361745199
Accuracy: 0.8130533329902156
Accuracy: 0.812317157164216
Accuracy: 

In [14]:
    model_classical.test()
    precision, recall, f1 = model_classical.calculate_metrics()

    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')

Average loss: 1.2165265975565445e-05
Accuracy: 0.9293736219406128
Average loss: 5.599295652312477e-05
Accuracy: 0.8976596891880035
Average loss: 0.00011995669068178026
Accuracy: 0.8777907490730286
Average loss: 0.0002744591987987117
Accuracy: 0.845419391989708
Average loss: 0.0003658200625511931
Accuracy: 0.8376261115074157
Average loss: 0.00042933832467874377
Accuracy: 0.8377913932005564
Average loss: 0.00042957857403075095
Accuracy: 0.8595461675098964
Average loss: 0.0004932053420527997
Accuracy: 0.8569128960371017
Average loss: 0.0005540268120168504
Accuracy: 0.8552648888693916
Average loss: 0.00057487230536403
Accuracy: 0.8604932844638824
Average loss: 0.0005764284688644339
Accuracy: 0.8708793466741388
Average loss: 0.0005995057927154959
Accuracy: 0.8735331892967224
Average loss: 0.0006636570407242355
Accuracy: 0.8707856856859647
Average loss: 0.001521614293366239
Accuracy: 0.8376498988696507
Average loss: 0.0015819885922033659
Accuracy: 0.8379840691884358
Average loss: 0.001618892

<ipython-input-7-7ffc81f5f320>:136: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  true_score_tensor = torch.tensor(self.true_score)


In [15]:
def print_tensor(tensor_arr):
  true_score_items = [float("{:.2f}".format(x.item())) for x in tensor_arr]
  start = 0
  end = len(true_score_items)
  step = 20
  for i in range(start, end, step):
      x = i
      print(true_score_items[x:x+step])

true_score_tensor = torch.tensor(model_classical.true_score)
pred_score_tensor = torch.tensor(model_classical.pred_score)

print("True values\n-----------")
print_tensor(true_score_tensor)
print("Pred values\n-----------")
print_tensor(pred_score_tensor)

True values
-----------
[0.09, 0.03, 0.0, 0.42, 0.36, 0.0, 0.15, 0.0, 0.01, 0.07, 0.14, 0.07, 0.0, 0.76, 0.32, 0.04, 0.0, 0.3, 0.12, 0.01]
[0.0, 0.05, 0.01, 0.13, 0.0, 0.21, 0.58, 0.01, 0.09, 0.02, 0.03, 0.07, 0.0, 0.39, 0.02, 0.0, 0.0, 0.1, 0.29, 0.07]
[0.03, 0.32, 0.13, 0.22, 0.06, 0.01, 0.01, 0.3, 0.35, 0.02, 0.07, 0.01, 0.01, 0.02, 0.33, 0.11, 0.0, 0.25, 0.37, 0.01]
[0.25, 0.24, 0.0, 0.62, 0.49, 0.02, 0.06, 0.05, 0.03, 0.0, 0.23, 0.61, 0.01, 0.6, 0.04, 0.0, 0.32, 0.67, 0.01, 0.47]
[0.44, 0.0, 0.02, 0.16, 0.0, 0.01, 0.38, 0.0, 0.65, 0.0, 0.04, 0.05, 0.02, 0.64, 0.0, 0.05, 1.0, 0.02, 0.04, 0.01]
[0.0, 0.02, 0.02, 0.24, 0.0, 0.13, 0.0, 0.0, 0.05, 0.04, 0.85, 0.01, 0.0, 0.08, 0.0, 0.0, 0.34, 0.19, 0.45, 0.28]
[0.01, 0.33, 0.23, 0.03, 0.0, 0.08, 0.0, 0.01, 0.31, 0.38, 0.24, 0.01, 0.02, 0.52, 0.23, 0.0, 0.46, 0.53, 0.18, 0.15]
[0.0, 0.0, 0.02, 0.06, 0.0, 0.41, 0.63, 0.0, 0.0, 0.41, 0.27, 0.36, 0.01, 0.01, 0.0, 0.4, 0.07, 0.07, 0.47, 0.73]
[0.18, 0.31, 0.0, 0.01, 0.77, 0.1, 0.2, 0.47, 0.5

In [17]:
model_gpt.train()

Epoch 1
-------------------------------
Evaluating with validation set
Accuracy: 0.7633202075958252
Accuracy: 0.550000011920929
Accuracy: 0.5455599427223206
Accuracy: 0.6499999910593033
Accuracy: 0.6726640343666077
Accuracy: 0.6333333353201548
Accuracy: 0.6052399788584027
Accuracy: 0.6250000074505806
Accuracy: 0.6596310999658372
Accuracy: 0.667335969209671
Accuracy: 0.6851527149027045
Accuracy: 0.672779972354571
Accuracy: 0.6874369107759916
Accuracy: 0.7071428554398673
Accuracy: 0.7108880122502644
Accuracy: 0.714165024459362
Accuracy: 0.7288212074952967
Accuracy: 0.7307378186119927
Accuracy: 0.7415768723738821
Accuracy: 0.7426640391349792
Accuracy: 0.7436476662045434
Accuracy: 0.7445418726314198
Accuracy: 0.7355043914007104
Accuracy: 0.7397200341025988
Accuracy: 0.7486640381813049
Accuracy: 0.7492277370049403
Accuracy: 0.7302444797975046
Accuracy: 0.7126171695334571
Accuracy: 0.7099986405208193
Accuracy: 0.707554680109024
Accuracy: 0.7093535680924693
Accuracy: 0.7110400255769491
Accura

In [18]:
  model_gpt.test()
  precision, recall, f1 = model_gpt.calculate_metrics()

  print(f'Precision: {precision}')
  print(f'Recall: {recall}')
  print(f'F1 Score: {f1}')

Average loss: 1.8697055965298558e-05
Accuracy: 0.9562227725982666
Average loss: 0.0020493235041546386
Accuracy: 0.75
Average loss: 0.006250575208673026
Accuracy: 0.6145924131075541
Average loss: 0.006830354204128791
Accuracy: 0.6499999910593033
Average loss: 0.012409601329893964
Accuracy: 0.5687554478645325
Average loss: 0.015427979959811015
Accuracy: 0.547925740480423
Average loss: 0.02100722708557619
Accuracy: 0.5044759597097125
Average loss: 0.02124533001447051
Accuracy: 0.5468886196613312
Average loss: 0.021276169017002713
Accuracy: 0.5909873578283522
Average loss: 0.02185594801245848
Accuracy: 0.607510894536972
Average loss: 0.022435727007914245
Accuracy: 0.6210301518440247
Average loss: 0.022466566010446446
Accuracy: 0.6479257444540659
Average loss: 0.023046345005902212
Accuracy: 0.6562562814125648
Average loss: 0.023626124001357978
Accuracy: 0.6633967416627067
Average loss: 0.02420590299681374
Accuracy: 0.6695851405461629
Average loss: 0.024785681992269507
Accuracy: 0.6749999895

In [19]:
true_score_tensor = torch.tensor(model_gpt.true_score)
pred_score_tensor = torch.tensor(model_gpt.pred_score)

print("True values\n-----------")
print_tensor(true_score_tensor)
print("Pred values\n-----------")
print_tensor(pred_score_tensor)

True values
-----------
[-0.6, 0.4, 0.8, -1.0, 1.0, 0.6, 1.0, -0.2, -0.4, -1.0, -1.0, -0.4, -1.0, -1.0, -1.0, -1.0, -0.6, -1.0, 0.4, 0.6]
[-1.0, 1.0, -1.0, 0.6, -1.0, -1.0, 0.4, -0.4, 0.4, -0.8, -1.0, -0.4, -0.8, -1.0, -1.0, 0.6, 0.2, -0.6, -0.6, -1.0]
[-0.8, -1.0, -0.6, -1.0, -1.0, -0.2, -1.0, -1.0, -1.0, -1.0, -0.4, 1.0, -1.0, -0.8, -1.0, 0.8, 0.2, -0.6, 0.6, -0.6]
[-0.4, -1.0, -1.0, -0.2, 0.8, -1.0, -0.4, -1.0, -1.0, -1.0, 0.2, -0.6, 0.8, 0.6, -0.2, -0.8, 0.0, 0.6, -1.0, -0.8]
[-0.8, -1.0, 0.6, -0.6, 0.6, -1.0, -1.0, 0.6, -0.6, -1.0, 0.4, 0.6, -0.4, -1.0, -0.6, -1.0, 0.8, -1.0, 0.6, -1.0]
[-0.4, -1.0, 0.2, -0.2, -0.6, -1.0, -1.0, -1.0, 0.4, -0.8, 1.0, -0.4, 0.2, -0.6, -0.6, 0.4, -1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, 0.4, -0.4, -1.0, -1.0, -1.0, 0.8, 0.2, -1.0, -1.0, 0.8, 1.0, 0.4, -1.0, -0.8, 0.6, 0.4, -1.0, -1.0]
[-1.0, -1.0, 0.8, -0.8, 0.4, -1.0, -0.6, 0.2, -0.8, -1.0, 0.6, -0.6, 1.0, 0.6, -1.0, -1.0, -0.4, -0.6, -1.0, -1.0]
[-1.0, -1.0, 1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 0.6, 